# Qualitative Review

### Required Modules

In [7]:
import os
import json
from IPython.display import display, Markdown
import random

from typing import Dict, Any, Union, List

### Load Results

In [ ]:
RULES_AND_STATS = os.path.join("..", "baselines", "results", "tfidf_bm25_lead_results.json")
TWO_PASS_LLM_SUMMARY = os.path.join("..", "baselines", "results", "llm_summarized_results.json")
LLM_EMBEDDING_SUMMARY = os.path.join("..", "baselines", "results", "llm_embeddings_results.json")


def load_json(filepath: str) -> Union[Dict[str, Any], List[Dict[str, Any]]]:
    """Return JSON data."""
    if os.path.exists(filepath):
        with open(filepath, "r") as f:
            data = json.load(f)
        return data


results = {}
rules_n_stats = load_json(RULES_AND_STATS)
for paper_id, data in rules_n_stats.items():
    results[paper_id] = {}

    # Article's abstract
    results[paper_id]["Abstract"] = ' '.join(data["abstract"]["plain"])

    # 1. Lead N prediction
    results[paper_id]["Lead N"] = data["lead_n"]

    # 2. Lead N by Section prediction
    results[paper_id]["Lead N by Section"] = data["lead_n_by_section"]

    # 3. TF-IDF prediction
    n = len(data["abstract"]["plain"])
    tfidf_indexes = data["rank"]["TF-IDF"][:n]
    tfidf_sentences = [data["sentences"]["plain"][i] for i in tfidf_indexes]
    results[paper_id]["TF-IDF"] = ' '.join(tfidf_sentences)

    # 4. BM25 prediction
    n = len(data["abstract"]["plain"])
    tfidf_indexes = data["rank"]["BM25"][:n]
    tfidf_sentences = [data["sentences"]["plain"][i] for i in tfidf_indexes]
    results[paper_id]["BM25"] = ' '.join(tfidf_sentences)


two_pass_llm = load_json(TWO_PASS_LLM_SUMMARY)
for data in two_pass_llm:
    paper_id = data["paper_id"]

    # 5. Two-pass LLM prediction
    results[paper_id]["Two-pass w/ Gemma 3"] = data["gemma3:1b"]["summary"]\


llm_embeddings = load_json(LLM_EMBEDDING_SUMMARY)
for data in llm_embeddings:
    paper_id = data["paper_id"]

    # 6. Embedding-based LLM prediction
    results[paper_id]["Embedding-based w/ Gemini 2.5"] = data["gemini-2.5-flash"]["summary"]
    results[paper_id]["Embedding-based w/ Llama 3.1"] = data["llama-3.1-8b-instant"]["summary"]
    results[paper_id]["Embedding-based w/ Longformer Encoder-Decoder"] = data["led"]["summary"]

### Review Predictions

In [16]:
METHODS = [
    "Lead N",
    "Lead N by Section",
    "TF-IDF",
    "BM25",
    "Two-pass w/ Gemma 3",
    "Embedding-based w/ Gemini 2.5",
    "Embedding-based w/ Llama 3.1",
    "Embedding-based w/ Longformer Encoder-Decoder"]


def print_random_prediction(results: Dict[str, str]) -> None:
    """Display a randomly chosen prediction against the abstract."""
    paper_id = random.choice(list(results.keys()))
    paper_results = results[paper_id]

    abstract = results[paper_id]["Abstract"]
    method_1 = random.choice(METHODS)
    method_2 = random.choice([m for m in METHODS if m != method_1])
    summary_1 = results[paper_id][method_1]
    summary_2 = results[paper_id][method_2]

    text = f"""
<br>

## Paper ID: `{paper_id}`

### Abstract
{abstract}

### {method_1}
{summary_1}

### {method_2}
{summary_2}
"""

    display(Markdown(text))


print_random_prediction(results)


<br>

## Paper ID: `2511.21570v1`

### Abstract
In an era marked by rapid technological advancements and complex global challenges, responsible foresight has emerged as an essential framework for policymakers aiming to navigate future uncertainties and shape the future. Responsible foresight entails the ethical anticipation of emerging opportunities and risks, with a focus on fostering proactive, sustainable, and accountable future design. This paper coins the term "responsible computational foresight", examining the role of human-centric artificial intelligence and computational modeling in advancing responsible foresight, establishing a set of foundational principles for this new field and presenting a suite of AI-driven foresight tools currently shaping it. AI, particularly in conjunction with simulations and scenario analysis, enhances policymakers’ ability to address uncertainty, evaluate risks, and devise strategies geared toward sustainable, resilient futures. However, responsible foresight extends beyond mere technical forecasting; it demands a nuanced understanding of the interdependencies within social, environmental, economic and political systems, alongside a commitment to ethical, long-term decision-making that supports human intelligence. We argue that AI will play a role as a supportive tool in responsible, human-centered foresight, complementing rather than substituting policymaker judgment to enable the proactive shaping of resilient and ethically sound futures. This paper advocates for the thoughtful integration of AI into foresight practices to empower policymakers and communities as they confront the grand challenges of the 21st century.

### BM25
This is further compounded by AI’s need for vast amounts of data and high energy demands, highlighting an important challenge for responsible foresight: developing ethical, sustainable, and context-sensitive AI systems that fully support human intelligence in future decision-making. Nevertheless, AI’s ability to analyze extensive datasets, model complex systems, and simulate alternative futures presents a promising pathway to support responsible foresight, enabling policymakers to gain insights that extend beyond what human cognition alone can offer. Responsible foresight [ uruena2021foresight ] will thus require technical tools like AI to close the decision loop, but also a deep understanding of the interconnected social, economic, and environmental systems that influence future outcomes, alongside a commitment to ethical and sustainable decision-making. Each approach contributes unique insights: Superforecasting harnesses the wisdom of skilled forecasters to generate highly accurate predictions; prediction markets leverage collective knowledge; world simulation creates virtual environments to capture the complexity of our social, economic and environmental systems; and simulation intelligence uses AI to design control strategies within simulation worlds providing meaningful insights on efficient and resilient pathways for the future. Incorporating responsible foresight into policymaking increasingly will involve the use of algorithms to analyze complex data, predict potential outcomes, and offer insight into a range of possible futures. These complementary methods—Superforecasting, prediction markets, world simulation, simulation intelligence, scenario-building, participatory futures, futures literacy and hybrid intelligence—form a powerful toolkit for responsible computational foresight. To realize this potential, AI in responsible foresight must be conceived as an assistive tool—a cognitive exoskeleton that enables policymakers to navigate complexity, envision a range of desirable futures, and critically assess the impacts of various decisions.

### Embedding-based w/ Gemini 2.5
This text introduces "responsible computational foresight," a field that leverages human-centric AI to empower policymakers. The central argument is that AI should *augment* human intelligence, not replace it, by helping explore a broad spectrum of future possibilities—including unexpected, unintended, and desirable ones—beyond mere precise predictions or historical extrapolations.

This approach integrates various computational tools like forecasting, complex systems simulation, AI-assisted scenario-building, and simulation intelligence, alongside human elements such as participatory futures and futures literacy. These tools, combined with a deep understanding of interconnected social, economic, and environmental systems and a commitment to ethical decision-making, strengthen our ability to anticipate and prepare for a range of outcomes.

Ultimately, responsible computational foresight aims to enhance ethical decision-making, build resilience against uncertainty, and enable society to navigate complexity with informed, adaptive, and inclusive strategies, thereby shaping sustainable and equitable paths for future generations.


## Observations

#### `2511.21570v1` Summary with BM25

> This is further compounded by AI’s need for vast amounts of data and high energy demands, highlighting an important challenge for responsible foresight: developing ethical, sustainable, and context-sensitive AI systems that fully support human intelligence in future decision-making.
> Nevertheless, AI’s ability to analyze extensive datasets, model complex systems, and simulate alternative futures presents a promising pathway to support responsible foresight, enabling policymakers to gain insights that extend beyond what human cognition alone can offer.
> Responsible foresight [ uruena2021foresight ] will thus require technical tools like AI to close the decision loop, but also a deep understanding of the interconnected social, economic, and environmental systems that influence future outcomes, alongside a commitment to ethical and sustainable decision-making.
> Each approach contributes unique insights: Superforecasting harnesses the wisdom of skilled forecasters to generate highly accurate predictions; prediction markets leverage collective knowledge; world simulation creates virtual environments to capture the complexity of our social, economic and environmental systems; and simulation intelligence uses AI to design control strategies within simulation worlds providing meaningful insights on efficient and resilient pathways for the future.
> Incorporating responsible foresight into policymaking increasingly will involve the use of algorithms to analyze complex data, predict potential outcomes, and offer insight into a range of possible futures.
> These complementary methods—Superforecasting, prediction markets, world simulation, simulation intelligence, scenario-building, participatory futures, futures literacy and hybrid intelligence—form a powerful toolkit for responsible computational foresight.
> To realize this potential, AI in responsible foresight must be conceived as an assistive tool—a cognitive exoskeleton that enables policymakers to navigate complexity, envision a range of desirable futures, and critically assess the impacts of various decisions.

Although the selected sentences are meaningful, the first sentence of the summary starts as a response to a missing sentence. The whole summary is disjointed and the 

by [Manuel Velarde](mailto:manuel@velarde.me)